<a href="https://colab.research.google.com/github/jha-vineet69/Car-Damage-Detection/blob/main/Car_Damage_Detection_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import Libraries

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import glob
import numpy as np
import h5py
import json
import pickle as pk
from urllib.request import urlretrieve

# other imports
import warnings
warnings.filterwarnings('ignore')

# keras imports
import tensorflow as tf
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

'2.4.0'

In [ ]:
#Loading Models for various checks
first_check = VGG16(weights='imagenet')
second_check = pk.load(open("/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/classifier.pickle", 'rb')) #damaged vs whole
third_check = pk.load(open("/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/classifier2.pickle", 'rb')) # Front Rear Side
fourth_check = pk.load(open("/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/classifier3.pickle", 'rb')) # Minor Moderate Severe

553467904/553467096 [==============================] - 5s 0us/step


In [ ]:
# Loading  valid categories for identifying cars using VGG16
with open('/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/vgg16_cat_list.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [ ]:
# shortlisting top 27 Categories that VGG16 stores for cars (Can be altered for less or more)
cat_list  = [k for k, v in cat_counter.most_common()[:27]]
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

In [ ]:
CLASS_INDEX = None
CLASS_INDEX_PATH = '/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/imagenet_class_index.json'

In [ ]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [ ]:
def prepare_img_224(img_path):
    # urlretrieve(img_path, 'image.jpg')
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

### First Check - Car or Not?

In [ ]:
def car_categories_check(img_224, model):
    print ("Validating that this is a picture of your car...")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print ("Car Check Passed!!!")
            print ("\n")
            return True 
    return False 

### Second Check - Damaged or Not?

In [ ]:
def car_damage_check(img_224, classifier):
    print ("Validating that damage exists...")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    feature = model.predict(img_224)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print ("Validation complete - proceeding to location and severity determination")
        print ("\n")
        return True 
    else:
        return False

### Third Check - Damage Location.

In [ ]:
def location_assessment(img_224, classifier):
    print ("Validating the damage area - Front, Rear or Side")
    base_model = first_check
    train_labels = ['Front', 'Rear', 'Side']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    feature = model.predict(img_224)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car is damaged at - " + train_labels[preds[0]])
    print ("Location assesment complete")
    print("\n")

### Fourth Check - Damage Severity.

In [ ]:
def severity_assessment(img_224, classifier):
    print ("Validating the Severity...")
    base_model = first_check
    train_labels = ['Minor', 'Moderate', 'Severe']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    feature = model.predict(img_224)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car damage impact is - " + train_labels[preds[0]])
    print ("Severity assessment complete")

### Engine

In [ ]:
def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)

            if g1 is False:
                print ("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            g2 = car_damage_check(img_224, second_check)

            if g2 is False:
                print ("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            x = location_assessment(img_224, third_check)
            y = severity_assessment(img_224, fourth_check)
            break
        except:
            print ("Image not accessible. Please try again.")
            break

In [ ]:
img_path1 = '/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/not-car.jpg'
img_path2 = '/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/undamaged-car.jpg'
img_path3 = '/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 7/front-damaged-car.jpg'

In [ ]:
engine(img_path1)

Validating that this is a picture of your car...
Are you sure this is a picture of your car? Please submit another picture of the damage.
Hint: Try zooming in/out, using a different angle or different lighting.


In [ ]:
engine(img_path2)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Are you sure that your car is damaged? Please submit another picture of the damage.
Hint: Try zooming in/out, using a different angle or different lighting.


In [ ]:
engine(img_path3)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Front
Location assesment complete


Validating the Severity...
Your Car damage impact is - Moderate
Severity assessment complete
